In [10]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import os

KEY =  b"key{the_aes_key}"
FLAG = b"crypto{aes_flag}"
PLAIN =b"Howdy! I'm Flowe"
flag_m0 = os.urandom(16)
flag_c0 = os.urandom(16)

In [11]:
class AesIge:
    def __init__(self, key):
        self.cipher = AES.new(key, AES.MODE_ECB)

    def encrypt(self, data, m0=flag_m0, c0=flag_c0):
        data = pad(data, 16, 'pkcs7')

        last_block_plaintext = m0
        last_block_ciphertext = c0
        result = b''
        for i in range(0, len(data), 16):
            block = data[i: i + 16]
            print(block)
            x = AesIge._xor_blocks(block, last_block_ciphertext)
            print(x)
            x = self.cipher.encrypt(x)
            print(x)
            x = AesIge._xor_blocks(x, last_block_plaintext)
            print(x)
            result += x

            last_block_plaintext = block
            last_block_ciphertext = x

        return result, m0, c0

    def decrypt(self, data, m0, c0):
        last_block_plaintext = m0
        last_block_ciphertext = c0
        result = b''

        for i in range(0, len(data), 16):
            block = data[i: i + 16]
            # m0 and c0 can be negated
            # pkcs7 padding oracle
            # can get stream data of flag enc
            # unk: flag
            # ciph = enc(pad(flag) ^ c0) ^ m0
            # flag = upad(dec(ciph ^ m0) ^ c0)
            # issue is the typical padding oracle now requires us to change the ciphertext to check for padding errors with this config
            
            # flag = upad(dec(ciph) ^ c) # find c that works with no m
            # start with 0x00*15 + 0xi

            # plain = upad(dec(ciph ^ m) ^ c)
            # plain = upad(dec(ciph ^ c0) ^ 0000)
            print(block)
            x = AesIge._xor_blocks(block, last_block_plaintext)
            print(x)
            x = self.cipher.decrypt(x)
            print(x)
            x = AesIge._xor_blocks(x, last_block_ciphertext)
            print(x)
            result += x

            last_block_ciphertext = block
            last_block_plaintext = x

        if AesIge._is_pkcs7_padded(result):
            return unpad(result, 16, 'pkcs7')
        else:
            return None

    @staticmethod
    def _is_pkcs7_padded(message):
        padding = message[-message[-1]:]
        return all(padding[i] == len(padding) for i in range(0, len(padding)))

    @staticmethod
    def _xor_blocks(a, b):
        return bytes([x ^ y for x, y in zip(a, b)])

def xor(a, b):
        return bytes([x ^ y for x, y in zip(a, b)])

def encrypt_flag():
    ciphertext, m0, c0 = AesIge(KEY).encrypt(FLAG)
    return {"ciphertext": ciphertext.hex(), "m0": m0.hex(), "c0": c0.hex()}


def send_msg(ciphertext, m0, c0):
    ciphertext = bytes.fromhex(ciphertext)
    m0 = bytes.fromhex(m0)
    c0 = bytes.fromhex(c0)
    if len(ciphertext) % 16 != 0:
        return {"error": "Data length must be a multiple of the blocksize!"}
    if len(c0) != 16 or len(m0) != 16:
        return {"error": "m0 and c0 must be 16 bytes long!"}

    plaintext = AesIge(KEY).decrypt(ciphertext, m0, c0)
    if plaintext is not None:
        return {"msg": "Message received"}
    else:
        return {"error": "Can't decrypt the message."}

In [1]:
from cryptolib import encrypt_flag
ct, m0, c0 = encrypt_flag()
ct, m0, c0

(b'\xdd-%\xc1\xe6\x16H\xb6\xa1\xcb\xab\x88Fn\x83.\x17X\x16;h\x16\\\xf8\xc5\xef\xd2\x1a\xf5h+\xd5',
 b'\xc8\xdd@\xbeT\xe5Y\x06\x88B5\x1cr }\xfd',
 b'\x1e\x1c\xb7\rJ\xaf\x8ft\xd8E\xe8\xf8_cj"')

In [12]:
b'\xdd-%\xc1\xe6\x16H\xb6\xa1\xcb\xab\x88Fn\x83.\x17X\x16;h\x16\\\xf8\xc5\xef\xd2\x1a\xf5h+\xd5'[16:]

b'\x17X\x16;h\x16\\\xf8\xc5\xef\xd2\x1a\xf5h+\xd5'

In [22]:
from cryptolib import pool
def test(i: int, found: bytes):
    from cryptolib import send_msg
    s0 = b'}n\xce}>\xc0\xf4\x1c\xeb)\x84\xc8\x00\x17YN'
    # ct = b'\xdd-%\xc1\xe6\x16H\xb6\xa1\xcb\xab\x88Fn\x83.'
    # m0 = b'\xc8\xdd@\xbeT\xe5Y\x06\x88B5\x1cr }\xfd'
    ct = b'\x17X\x16;h\x16\\\xf8\xc5\xef\xd2\x1a\xf5h+\xd5' # ct1
    m0 = b"crypto{h3ll0_t3l3gr4m}" # m0
    payload = bytes([found[i]^(len(found)+1) for i in range(len(found))])
    return list(send_msg(ct.hex(), m0.hex(), (b'\x00'*(15-len(found)) + bytes([i])+payload).hex()).values())[0]=='Message received'
def get_vals():
    s0 = b"" #246, 149, 155, 198
    c0 = b'\xdd-%\xc1\xe6\x16H\xb6\xa1\xcb\xab\x88Fn\x83.' # ct0
    key = b""
    for j in range(16):
        _c0 = pool(test, 256, 16, [s0]*256).index(True)
        s0 = bytes([(_c0 ^ (len(s0)+1))]) + s0
        key = bytes([((_c0 ^ (len(s0))) ^ c0[-len(s0)])]) + key
        print(s0, key)
    return s0, key


In [23]:
get_vals()

  0%|          | 0/256 [00:00<?, ?it/s]

b'$' b'\n'


  0%|          | 0/256 [00:00<?, ?it/s]

b'\x89$' b'\n\n'


  0%|          | 0/256 [00:00<?, ?it/s]

...

  0%|          | 0/256 [00:00<?, ?it/s]

b'JW\xf5\x8bkB\xbc\xab\xc1\xa1\x82Ld\x89$' b'gr4m}\n\n\n\n\n\n\n\n\n\n'


  0%|          | 0/256 [00:00<?, ?it/s]

b'\xeeJW\xf5\x8bkB\xbc\xab\xc1\xa1\x82Ld\x89$' b'3gr4m}\n\n\n\n\n\n\n\n\n\n'


(b'\xeeJW\xf5\x8bkB\xbc\xab\xc1\xa1\x82Ld\x89$', b'3gr4m}\n\n\n\n\n\n\n\n\n\n')

In [66]:
chr((113 ^ 1) ^ c0[-1])

'}'

In [39]:
[test(i) for i in range(256)].index(True) ^ 2

b'~\xb2\x9e\xba\x9b\xd4A\xe3\xe7{0G\x88>\x82&'
b'\xb6\x9d\xaf\x99V\xc2H2\xfe\xb6\x84\xcay\x8d\xbb\xc7'
b'\x95\xa7\xa1\xd1\x1f\xbf\xe9.\xea\xa8\xa6\xcb*Nep'
b'\x95\xa7\xa1\xd1\x1f\xbf\xe9.\xea\xa8\xa6\xcb*Ne\x02'
b'~\xb2\x9e\xba\x9b\xd4A\xe3\xe7{0G\x88>\x82&'
b'\xb6\x9d\xaf\x99V\xc2H2\xfe\xb6\x84\xcay\x8d\xbb\xc7'
b'\x95\xa7\xa1\xd1\x1f\xbf\xe9.\xea\xa8\xa6\xcb*Nep'
b'\x95\xa7\xa1\xd1\x1f\xbf\xe9.\xea\xa8\xa6\xcb*Nd\x02'
b'~\xb2\x9e\xba\x9b\xd4A\xe3\xe7{0G\x88>\x82&'
b'\xb6\x9d\xaf\x99V\xc2H2\xfe\xb6\x84\xcay\x8d\xbb\xc7'
b'\x95\xa7\xa1\xd1\x1f\xbf\xe9.\xea\xa8\xa6\xcb*Nep'
b'\x95\xa7\xa1\xd1\x1f\xbf\xe9.\xea\xa8\xa6\xcb*Ng\x02'
b'~\xb2\x9e\xba\x9b\xd4A\xe3\xe7{0G\x88>\x82&'
b'\xb6\x9d\xaf\x99V\xc2H2\xfe\xb6\x84\xcay\x8d\xbb\xc7'
b'\x95\xa7\xa1\xd1\x1f\xbf\xe9.\xea\xa8\xa6\xcb*Nep'
b'\x95\xa7\xa1\xd1\x1f\xbf\xe9.\xea\xa8\xa6\xcb*Nf\x02'
b'~\xb2\x9e\xba\x9b\xd4A\xe3\xe7{0G\x88>\x82&'
b'\xb6\x9d\xaf\x99V\xc2H2\xfe\xb6\x84\xcay\x8d\xbb\xc7'
b'\x95\xa7\xa1\xd1\x1f\xbf\xe9.\xea\xa8\xa6\xcb*N

101

'g'

In [23]:
xor(AesIge(KEY).encrypt(FLAG)[0][:16], m0) # b"crypto{aaaaaaaaa"

b'crypto{aes_flag}'
b'\xa5\xf5\x19j\xd2s\xa0\x87\x00\x1eY0\xf0\xa6\xf8\x0b'
b'M\xfd\xbc\x93X\x13\x00\xe2\xc6gp\x87\xf3\x03=u'
b'\xc5Y}v\xfc\x91.\xdfU\x94<}\x17Z\xc6\x02'
b'\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'
b'\xd5Imf\xec\x81>\xcfE\x84,m\x07J\xd6\x12'
b'\x02\xb3\xd8\xd61h\xf6=V\x06m\x0f\xe3"\xe5/'
b'a\xc1\xa1\xa6E\x07\x8d\\3u2i\x8fC\x82R'


b'M\xfd\xbc\x93X\x13\x00\xe2\xc6gp\x87\xf3\x03=u'

In [22]:
b'M\xfd\xbc\x93X\x13\x00\xe2\xc6gp\x87\xf3\x03=u'

b'M\xfd\xbc\x93X\x13\x00\xe2\xc6gp\x87\xf3\x03=u'

In [19]:
AesIge(KEY).decrypt(ct, m0, c0)

b'\xc5Y}v\xfc\x91.\xdfU\x94<}\x17Z\xc6\x02'
b'M\xfd\xbc\x93X\x13\x00\xe2\xc6gp\x87\xf3\x03=u'
b'\xa5\xf5\x19j\xd2s\xa0\x87\x00\x1eY0\xf0\xa6\xf8\x0b'
b'crypto{aes_flag}'
b'a\xc1\xa1\xa6E\x07\x8d\\3u2i\x8fC\x82R'
b'\x02\xb3\xd8\xd61h\xf6=V\x06m\x0f\xe3"\xe5/'
b'\xd5Imf\xec\x81>\xcfE\x84,m\x07J\xd6\x12'
b'\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'


b'crypto{aes_flag}'

In [3]:
encrypt_flag()

b'crypto{aes_flag}'
b'\xa5\xf5\x19j\xd2s\xa0\x87\x00\x1eY0\xf0\xa6\xf8\x0b'
b'M\xfd\xbc\x93X\x13\x00\xe2\xc6gp\x87\xf3\x03=u'
b'\xc5Y}v\xfc\x91.\xdfU\x94<}\x17Z\xc6\x02'
b'\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'
b'\xd5Imf\xec\x81>\xcfE\x84,m\x07J\xd6\x12'
b'\x02\xb3\xd8\xd61h\xf6=V\x06m\x0f\xe3"\xe5/'
b'a\xc1\xa1\xa6E\x07\x8d\\3u2i\x8fC\x82R'


{'ciphertext': 'c5597d76fc912edf55943c7d175ac60261c1a1a645078d5c337532698f438252',
 'm0': '88a4c1e5a4822e3d93f34cfae459fb77',
 'c0': 'c687601aa61cdbe6656d06569cc79f76'}

In [14]:
ct = b'\xdd-%\xc1\xe6\x16H\xb6\xa1\xcb\xab\x88Fn\x83.\x17X\x16;h\x16\\\xf8\xc5\xef\xd2\x1a\xf5h+\xd5'
m0 = bytes.fromhex("ecd294c59a28482fb5aeae477f84af8f")
c0 = bytes.fromhex("80c305eaa136e6fcda9378f2e08bfc56")
send_msg(ct[:16].hex(), m0.hex(), (b'\x00'*(15) + bytes([162])).hex())

b'\xdd-%\xc1\xe6\x16H\xb6\xa1\xcb\xab\x88Fn\x83.'
b'1\xff\xb1\x04|>\x00\x99\x14e\x05\xcf9\xea,\xa1'
b'v\xf3\xac@\x18BM\x07\nUM\xf7k\xcc[\xa3'
b'v\xf3\xac@\x18BM\x07\nUM\xf7k\xcc[\x01'


{'msg': 'Message received'}

In [62]:
chr(163 ^ c0[0])

'#'

In [56]:
b'\xa3'[0] ^ 1

162

In [33]:
bytes.fromhex("80c305eaa136e6fcda9378f2e08bfc56")[0]

128

In [34]:
(6 ^ 1) ^ 128

135

In [30]:
chr(81)

'Q'